In [1]:
import pandas as pd
from datetime import date

# Define the file paths
main_file_path = r'C:\Users\Jackson Tengeya\Desktop\Recon\py\Bloom 2.0\Bloom 2 Dataset_Batch 2.csv'
info_file_path = r'C:\Users\Jackson Tengeya\Desktop\Recon\py\Bloom 2.0\Daily Payment Information Leo.csv'
safaricom_balances_path = r'C:\Users\Jackson Tengeya\Desktop\Recon\py\Bloom 2.0\safaricom_balances.csv'

# Read both CSV files into DataFrames
main_df = pd.read_csv(main_file_path)
info_df = pd.read_csv(info_file_path)

# Read the "safaricom_balances.csv" file
safaricom_balances_df = pd.read_csv(safaricom_balances_path)

# Convert the "Account Number" column to the same data type in both DataFrames (if not already)
main_df['Account Number'] = main_df['Account Number'].astype(str)
info_df['Account Number'] = info_df['Account Number'].astype(str)

# Create a dictionary to map the columns from the main file to the info file
column_mapping = {
    'Surname': 'Surname',
    'Forename 1': 'Forename 1',
    'Forename 2': 'Forename 2',
    'Forename 3': 'Forename 3',
    'Primary Identification Document Type': 'Primary Identification Document Type',
    'Primary Identification Document Number': 'Primary Identification Document Number',
    'Currency of Facility': 'Currency of Facility',
    'Original Amount': 'Original Amount',
    'Current Balance': 'Current Balance',  # Keep this mapping for other cases
    'Number of Instalments in Arrears': 'Number of Instalments in Arrears',
    'Number of Days in Arrears': 'Number of Days in Arrears',
    'Account Status': 'Account Status',
}

# Iterate through the rows of the main DataFrame
for index, row in main_df.iterrows():
    account_number = row['Account Number']
    # Find the corresponding row(s) in the info DataFrame
    matching_rows = info_df[info_df['Account Number'] == account_number]
    if not matching_rows.empty:
        # Update the existing row(s) in the info DataFrame with data from the main DataFrame
        for col_main, col_info in column_mapping.items():
            info_df.loc[matching_rows.index, col_info] = row[col_main]

# Set the 'Snapshot Date', 'Client Type', and 'Company Name' columns to the desired values
today_date = date.today().strftime('%Y%m%d')
info_df['Snapshot Date'] = today_date
info_df['Client Type'] = 'A'
info_df['Company Name'] = 'AS'

# Convert the 'loan_mifos_id' column in 'safaricom_balances_df' to the same data type as 'Account Number'
safaricom_balances_df['loan_mifos_id'] = safaricom_balances_df['loan_mifos_id'].astype(str)

# Merge the dataframes based on 'Account Number'
merged_df = info_df.merge(safaricom_balances_df[['loan_mifos_id', 'safaricom_loan_balance']], 
                        left_on='Account Number', right_on='loan_mifos_id', how='left')

# Multiply 'Original Amount', 'Payment Amount', and 'safaricom_loan_balance' by 100
columns_to_multiply = ['Original Amount', 'Payment Amount', 'safaricom_loan_balance']
for col in columns_to_multiply:
    merged_df[col] = merged_df[col] * 100

# Format "Current Balance" with leading zeros
merged_df['Current Balance'] = merged_df['safaricom_loan_balance'].apply(lambda x: f'{int(x):03d}' if not pd.isna(x) else '')

# Format "Payment Date" to YYYYMMDD format
merged_df['Payment Date'] = pd.to_datetime(merged_df['Payment Date'], format='%d/%m/%Y').dt.strftime('%Y%m%d')

# Define the output file path for the updated data in XLSX format
output_file_path = r'C:\Users\Jackson Tengeya\Desktop\Recon\py\Bloom 2.0\Bloom 2.0 Daily_Payment_Information_06112023.xlsx'

# Save the updated data to an XLSX file, keeping only the original columns from info_file_path
output_columns = info_df.columns
merged_df = merged_df[output_columns]
merged_df.to_excel(output_file_path, index=False)

# Display a confirmation message
print(f'Updated data saved to {output_file_path}')


Updated data saved to C:\Users\Jackson Tengeya\Desktop\Recon\py\Bloom 2.0\Bloom 2.0 Daily_Payment_Information_06112023.xlsx
